In [ ]:
!pip install pymupdf

In [ ]:
!pip install openai

In [ ]:
openai_api_key = input("Enter your OpenAI API key: ");

In [ ]:
from openai import OpenAI
import os

client = OpenAI(
    # This is the default and can be omitted
    api_key=openai_api_key,
)

def query_gpt4(prompt, input, temperature=0.2):
    """
    Sends a prompt to GPT-4 and retrieves the response with a specified temperature.

    Parameters:
        prompt (str): The input prompt for GPT-4.
        temperature (float): Controls the randomness in the response, lower is more deterministic.

    Returns:
        str: The response from GPT-4.
    """

    try:
        response = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": prompt,
                },
                {
                    "role": "user",
                    "content": input
                }
            ],
            model="gpt-4-turbo",
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [ ]:
import fitz

In [ ]:
pdf_path = "DDIA.pdf"

In [ ]:
doc = fitz.open(pdf_path)

In [ ]:
start_page, end_page = 8, 13

In [ ]:
pages = [doc.load_page(i) for i in range(start_page, end_page + 1)]

In [ ]:
raw_texts = '\n'.join([str(page.get_text()) for page in pages])

In [ ]:
raw_texts

In [ ]:
bookSchema = """
type: object
properties:
  id:
    type: string
    description: The ID of the textbook.
  title:
    type: string
    description: The title of the textbook.
  chapters:
    type: array
    description: A list of chapters in the textbook.
    items:
      id:
        type: string
        description: The ID of the chapter.
  metadata:
    type: object
    additionalProperties: true
    description: Metadata for the textbook.

required:
  - id
  - title
  - chapters"""

chapterSchema = """
type: object
properties:
  id:
    type: string
    description: The ID of the chapter.
  title:
    type: string
    description: The title of the chapter.
  range:
    type: array
    items:
      type: integer
    minItems: 2
    maxItems: 2
    description: The start and end pages of the chapter.
  subchapters:
    type: array
    items:
      id:
        type: string
        description: The ID of the subchapter.
    description: A list of subchapters.
  metadata:
    type: object
    additionalProperties: true
    description: Metadata for the chapter.

required:
  - id
  - title
  - range"""

In [ ]:
cleaning_prompt_template = f"""
You are given as input the table of contents of Designing Data-Intensive Applications. The text is scraped from a PDF file, so there are headers/footers like "table of contents" you have to ignore.
Help me to clean this table of contents into a list of major chapters and subchapters, which may have subchapters of their own, each pointing to a page number.
Chapters that aren't technical content like "Preface" or "Glossary" should be omitted.

Clean it into the following YAML schema:
Book shema:
{bookSchema}

Chapter schema:
{chapterSchema}

Don't worry about generating the IDs for now.
"""

In [ ]:
precleaning_prompt_template = f"""
You are given as input the table of contents of Designing Data-Intensive Applications. The text is scraped from a PDF file, so there are headers/footers like "table of contents" you have to ignore.
Help me to clean this table of contents into a list of major chapters and subchapters.
Chapters that aren't technical content like "Preface" or "Glossary" should be omitted.

Print into the following example yaml format, always reproducing the page number following the chapter name.
Make sure that the indentation is correct.

If a chapter starts on page 107, it should read

chapter name 107

Example:

- Major chapter name 1
  - subchapter name 1
    - subsubchapter name 3
    - subsubchapter name 4
  - subchapter name 6
  - subchapter name 7
- Major chapter name 9
  - subchapter name 10
  - subchapter name 12"""

In [ ]:
test = query_gpt4("hi", "hi there")
print(test)

In [ ]:
cleaned = query_gpt4(precleaning_prompt_template, raw_texts)

In [ ]:
cleaned

In [ ]:
cleaned = cleaned.replace("```yaml\n", "")
cleaned = cleaned.replace("```", "")

In [ ]:
cleaned

In [ ]:
old_cleaned = """
- "Foundations of Data Systems 3"
  - "Reliable, Scalable, and Maintainable Applications 3"
    - "Thinking About Data Systems 4"
    - Reliability 6
    - Hardware Faults 7
    - Software Errors 8
    - Human Errors 9
    - How Important Is Reliability? 10
    - Scalability 10
    - Describing Load 11
    - Describing Performance 13
    - Approaches for Coping with Load 17
    - Maintainability 18
    - Operability: Making Life Easy for Operations 19
    - Simplicity: Managing Complexity 20
    - Evolvability: Making Change Easy 21
  - Summary 22
  - Data Models and Query Languages 27
    - Relational Model Versus Document Model 28
    - The Birth of NoSQL 29
    - The Object-Relational Mismatch 29
    - Many-to-One and Many-to-Many Relationships 33
    - Are Document Databases Repeating History? 36
    - Relational Versus Document Databases Today 38
    - Query Languages for Data 42
    - Declarative Queries on the Web 44
    - MapReduce Querying 46
    - Graph-Like Data Models 49
    - Property Graphs 50
    - The Cypher Query Language 52
    - Graph Queries in SQL 53
    - Triple-Stores and SPARQL 55
    - The Foundation: Datalog 60
  - Summary 63
  - Storage and Retrieval 69
    - Data Structures That Power Your Database 70
    - Hash Indexes 72
    - SSTables and LSM-Trees 76
    - B-Trees 79
    - Comparing B-Trees and LSM-Trees 83
    - Other Indexing Structures 85
    - Transaction Processing or Analytics? 90
    - Data Warehousing 91
    - Stars and Snowflakes: Schemas for Analytics 93
    - Column-Oriented Storage 95
    - Column Compression 97
    - Sort Order in Column Storage 99
    - Writing to Column-Oriented Storage 101
    - Aggregation: Data Cubes and Materialized Views 101
  - Summary 103
  - Encoding and Evolution 111
    - Formats for Encoding Data 112
    - Language-Specific Formats 113
    - JSON, XML, and Binary Variants 114
    - Thrift and Protocol Buffers 117
    - Avro 122
    - The Merits of Schemas 127
    - Modes of Dataflow 128
    - Dataflow Through Databases 129
    - Dataflow Through Services: REST and RPC 131
    - Message-Passing Dataflow 136
  - Summary 139
- Distributed Data 151
  - Replication 151
    - Leaders and Followers 152
    - Synchronous Versus Asynchronous Replication 153
    - Setting Up New Followers 155
    - Handling Node Outages 156
    - Implementation of Replication Logs 158
    - Problems with Replication Lag 161
    - Reading Your Own Writes 162
    - Monotonic Reads 164
    - Consistent Prefix Reads 165
    - Solutions for Replication Lag 167
    - Multi-Leader Replication 168
    - Use Cases for Multi-Leader Replication 168
    - Handling Write Conflicts 171
    - Multi-Leader Replication Topologies 175
    - Leaderless Replication 177
    - Writing to the Database When a Node Is Down 177
    - Limitations of Quorum Consistency 181
    - Sloppy Quorums and Hinted Handoff 183
    - Detecting Concurrent Writes 184
  - Summary 192
  - Partitioning 199
    - Partitioning and Replication 200
    - Partitioning of Key-Value Data 201
    - Partitioning by Key Range 202
    - Partitioning by Hash of Key 203
    - Skewed Workloads and Relieving Hot Spots 205
    - Partitioning and Secondary Indexes 206
    - Partitioning Secondary Indexes by Document 206
    - Partitioning Secondary Indexes by Term 208
    - Rebalancing Partitions 209
    - Strategies for Rebalancing 210
    - Operations: Automatic or Manual Rebalancing 213
    - Request Routing 214
    - Parallel Query Execution 216
  - Summary 216
  - Transactions 221
    - The Slippery Concept of a Transaction 222
    - The Meaning of ACID 223
    - Single-Object and Multi-Object Operations 228
    - Weak Isolation Levels 233
    - Read Committed 234
    - Snapshot Isolation and Repeatable Read 237
    - Preventing Lost Updates 242
    - Write Skew and Phantoms 246
    - Serializability 251
    - Actual Serial Execution 252
    - Two-Phase Locking (2PL) 257
    - Serializable Snapshot Isolation (SSI) 261
  - Summary 266
  - The Trouble with Distributed Systems 273
    - Faults and Partial Failures 274
    - Cloud Computing and Supercomputing 275
    - Unreliable Networks 277
    - Network Faults in Practice 279
    - Detecting Faults 280
    - Timeouts and Unbounded Delays 281
    - Synchronous Versus Asynchronous Networks 284
    - Unreliable Clocks 287
    - Monotonic Versus Time-of-Day Clocks 288
    - Clock Synchronization and Accuracy 289
    - Relying on Synchronized Clocks 291
    - Process Pauses 295
    - Knowledge, Truth, and Lies 300
    - The Truth Is Defined by the Majority 300
    - Byzantine Faults 304
    - System Model and Reality 306
  - Summary 310
  - Consistency and Consensus 321
    - Consistency Guarantees 322
    - Linearizability 324
    - What Makes a System Linearizable? 325
    - Relying on Linearizability 330
    - Implementing Linearizable Systems 332
    - The Cost of Linearizability 335
    - Ordering Guarantees 339
    - Ordering and Causality 339
    - Sequence Number Ordering 343
    - Total Order Broadcast 348
    - Distributed Transactions and Consensus 352
    - Atomic Commit and Two-Phase Commit (2PC) 354
    - Distributed Transactions in Practice 360
    - Fault-Tolerant Consensus 364
    - Membership and Coordination Services 370
  - Summary 373
- Derived Data 389
  - Batch Processing 389
    - Batch Processing with Unix Tools 391
    - Simple Log Analysis 391
    - The Unix Philosophy 394
    - MapReduce and Distributed Filesystems 397
    - MapReduce Job Execution 399
    - Reduce-Side Joins and Grouping 403
    - Map-Side Joins 408
    - The Output of Batch Workflows 411
    - Comparing Hadoop to Distributed Databases 414
    - Beyond MapReduce 419
    - Materialization of Intermediate State 419
    - Graphs and Iterative Processing 424
    - High-Level APIs and Languages 426
  - Summary 429
  - Stream Processing 439
    - Transmitting Event Streams 440
    - Messaging Systems 441
    - Partitioned Logs 446
    - Databases and Streams 451
    - Keeping Systems in Sync 452
    - Change Data Capture 454
    - Event Sourcing 457
    - State, Streams, and Immutability 459
    - Processing Streams 464
    - Uses of Stream Processing 465
    - Reasoning About Time 468
    - Stream Joins 472
    - Fault Tolerance 476
  - Summary 479
  - The Future of Data Systems 489
    - Data Integration 490
    - Combining Specialized Tools by Deriving Data 490
    - Batch and Stream Processing 494
    - Unbundling Databases 499
    - Composing Data Storage Technologies 499
    - Designing Applications Around Dataflow 504
    - Observing Derived State 509
    - Aiming for Correctness 515
    - The End-to-End Argument for Databases 516
    - Enforcing Constraints 521
    - Timeliness and Integrity 524
    - Trust, but Verify 528
    - Doing the Right Thing 533
    - Predictive Analytics 533
    - Privacy and Tracking 536
  - Summary 543"""

In [ ]:
class Chapter:
    def __init__(self, title, firstPage, subchapters):
        self.id = id
        self.title = title
        self.firstPage = firstPage
        self.subchapters = subchapters

    def setLastPage(self, lastPage):
        self.lastPage = lastPage
        self.range = [self.firstPage, lastPage]



In [ ]:
def parse(str_list, index = 0):
    """
    Input: list of strings in a yaml-like format
    Output: (list of Chapter objects, index of last chapter)
    """
    def parse_chapter_info(string):
        """
        returns (title, firstPage, indentation)
        """
        # print("parsing", string)
        indentation = 0
        while string[indentation] == " ":
            indentation += 1

        number_start = len(string) - 1
        while string[number_start-1].isnumeric():
            number_start -= 1
        title = string[indentation+2:number_start].strip()
        # print(string, number_start, string[number_start:].strip())
        firstPage = int(string[number_start:].strip())
        return title, firstPage, indentation

    chapters = []
    while index < len(str_list):
        string = str_list[index]
        title, firstPage, indentation = parse_chapter_info(string)
        index += 1
        if index < len(str_list) and parse_chapter_info(str_list[index])[2] > indentation:
            subchapters, index = parse(str_list, index)
        else:
            subchapters = []
        chapters.append(Chapter(title, firstPage, subchapters))
        if index < len(str_list) and parse_chapter_info(str_list[index])[2] < indentation:
            break
    return chapters, index



cleanedList = list(filter(lambda x: x != "", cleaned.split('\n')))
chapters, index = parse(cleanedList)
print(chapters)

In [ ]:
for chapter in chapters:
    print(chapter.title, chapter.firstPage)

In [ ]:
def setPageRanges(chapters, lastPage=543):
    for idx, chapter in reversed(list(enumerate(chapters))):
        chapter.setLastPage(lastPage)
        lastPage = chapters[idx].firstPage
        setPageRanges(chapter.subchapters, lastPage)

setPageRanges(chapters)

In [ ]:
for chapter in chapters:
    print(chapter.title, chapter.range)
for subchapter in chapters[0].subchapters:
    print(subchapter.title, subchapter.range)

In [ ]:
BOOK_OFFSET = 22

# Print the parsed AST to YAML!!!

In [ ]:
!pip install shortuuid

In [ ]:
import shortuuid
import yaml

In [ ]:
def random_uuid():
    return shortuuid.uuid()[:6]

def chapter_id():
    return "chapter-" + random_uuid()

def book_id():
    return "book-" + random_uuid()

def subchapter_id(subchapter):
    return random_uuid()

book_id()

In [ ]:
bookDict = {
    "id": book_id(),
    "title": "Designing Data-Intensive Applications",
    "chapters": [],
    "metadata": {
        "authors": ["Martin Kleppmann"],
        "publisher": "Springer",
        "date": "2019",
        "stars": 1,
        "image": "https://m.media-amazon.com/images/I/91JAIKQUkYL._AC_UF1000,1000_QL80_.jpg"
    }
}


In [ ]:
def getChapterDicts(chapters, bookFrom, depth=1):
    chapterDicts = []
    for chapter in chapters:
        subChapterDicts = getChapterDicts(chapter.subchapters, bookFrom, depth+1)
        chapterDict = {
            "id": chapter_id(),
            "title": chapter.title,
            "range": [chapter.range[0] + BOOK_OFFSET, chapter.range[1] + BOOK_OFFSET],
            "subchapters": [subDict["id"] for subDict in subChapterDicts if subDict["metadata"]["depth"] == depth + 1],
            "metadata": {
                "book": bookFrom,
                "depth": depth
            }
        }
        chapterDicts.append(chapterDict)
        chapterDicts.extend(subChapterDicts)
    return chapterDicts

chapterDicts = getChapterDicts(chapters, bookDict["id"])

In [ ]:
bookDict["chapters"] = [chapterDict["id"] for chapterDict in chapterDicts if chapterDict["metadata"]["depth"] == 1]

In [ ]:
print(yaml.dump(bookDict))

In [ ]:
print(yaml.dump(chapterDicts))